In [ ]:
import os
os.environ['THEANO_FLAGS']='mode=FAST_RUN,device=gpu0,floatX=float32'

In [1]:
import theano
import theano.tensor as T
import theano.tensor.nnet as NN
import numpy as NP
import numpy.random as RNG
import h5py
import dataUtils
from collections import OrderedDict
from Wrapper import Model, rmsprop

In [2]:
def get_conv_out(img_shape, conv_shape):
    img_row, img_col = img_shape
    nr_filters, nr_channels, filter_row, filter_col, stride_row, stride_col = conv_shape
    output_dim = ((img_row - filter_row) / stride_row + 1) * ((img_col - filter_col) / stride_col + 1) * nr_filters * nr_channels
    return output_dim

In [14]:
def _step(cur_in):
    
    conv1_shape = (32, 1, 5, 5, 1, 1)
    conv1_h = model.conv(cur_in = cur_in, name = 'conv1', shape=conv1_shape)
    conv1_out_dim = get_conv_out(img_shape, conv1_shape)
    fc1 = model.fc(cur_in = conv1_h.flatten(2), name = 'fc1', shape=(conv1_out_dim, out_dim))
    return fc1
    
img_shape = (28, 28)
out_dim = 10
model = Model()
img_in = T.tensor4()
seg_img = T.tensor4()
img_tar = T.matrix()
img_out = _step(img_in)
_EPSI = 1e-6
sal_img = T.grad(T.sum(img_out[:,0]), img_in)
cost2 = T.mean((sal_img-seg_img)**2)
cost = cost2 + T.mean(T.sum(NN.binary_crossentropy(T.clip(img_out, _EPSI, 1.0 - _EPSI), img_tar), axis=1))

train_func = theano.function([img_in, img_tar, seg_img], [cost, img_out], updates=rmsprop(cost, model.weightsPack.getW_list()), allow_input_downcast=True)
predict_func = theano.function([img_in], img_out, allow_input_downcast=True)

In [ ]:
''' 
load data 
'''
in_dim = 49
out_dim = 49
batch_size = 16

X_train, Y_train=dataUtils.read_txt('ptb.train.txt')
#X_test, Y_test=dataUtils.read_txt('ptb.test.txt')


In [ ]:
def _step(trash, cur_in, rec_in1, rec_in2, rec_in3):
    
    gru1_h = model.gru(cur_in = cur_in, rec_in = rec_in1, name='gru1', shape=(in_dim, 20))
    gru2_h = model.gru(cur_in = gru1_h, rec_in = rec_in2, name='gru2', shape=(20, 20))
    gru3_h = model.gru(cur_in = gru2_h, rec_in = rec_in3, name='gru3', shape=(20, 20))
    merge = T.concatenate([gru1_h, gru2_h, gru3_h], axis=1)
    fc1 = model.fc(cur_in = merge, name='fc1', shape=(60, out_dim))
    return [fc1] + [gru1_h, gru2_h, gru3_h]
    
model = Model()
char_in = T.tensor3()
char_tar = T.tensor3()
_char_in = char_in.dimshuffle(1, 0, 2)
starts = T.matrix()

sc, _ = theano.scan(_step, sequences=[_char_in], outputs_info=[starts, T.zeros((char_in.shape[0], 20)), T.zeros((char_in.shape[0], 20)), T.zeros((char_in.shape[0], 20))])
char_out = sc[0].dimshuffle(1, 0, 2)
_EPSI = 1e-6
cost = T.mean(T.sum(NN.binary_crossentropy(T.clip(char_out, _EPSI, 1.0 - _EPSI), char_tar), axis=2))

#cost = T.mean(T.sum((char_tar - char_out) ** 2, axis=2))
train_func = theano.function([char_in, starts, char_tar], [cost, char_out], updates=rmsprop(cost, model.weightsPack.getW_list()), allow_input_downcast=True)
predict_func = theano.function([char_in, starts], char_out, allow_input_downcast=True)

In [ ]:
def _step(cur_in, trash, rec_in1, rec_in2, rec_in3):
    
    gru1_h = model2.gru(cur_in = cur_in, rec_in = rec_in1, name='gru1', shape=(in_dim, 100))
    gru2_h = model2.gru(cur_in = gru1_h, rec_in = rec_in2, name='gru2', shape=(100, 100))
    gru3_h = model2.gru(cur_in = gru2_h, rec_in = rec_in3, name='gru3', shape=(100, 100))
    merge = T.concatenate([gru1_h, gru2_h, gru3_h], axis=1)
    fc1 = model.fc(cur_in = merge, name='fc1', shape=(60, out_dim))
    return [fc1] + [gru1_h, gru2_h, gru3_h]
    
model2 = Model()


char_in = T.tensor3()
char_tar = T.tensor3()
_char_in = char_in.dimshuffle(1, 0, 2)
starts = T.matrix()

sc, _ = theano.scan(_step, sequences=[_char_in], outputs_info=[starts, T.zeros((char_in.shape[0], 20)), T.zeros((char_in.shape[0], 20)), T.zeros((char_in.shape[0], 20))])
char_out = sc[0].dimshuffle(1, 0, 2)

#cost = T.mean(T.sum((char_tar - char_out) ** 2, axis=2))

#train_func = theano.function([char_in, starts, char_tar], [cost, char_out], updates=rmsprop(cost, model.weightsPack.getW_list()), allow_input_downcast=True)
#Jpredict_func = theano.function([char_in, starts], char_out, allow_input_downcast=True)
jacobi = T.jacobian(char_out[0, :50, 0:3].flatten(), char_in)
jacobi_func = theano.function([char_in, starts], jacobi, allow_input_downcast=True)

In [ ]:
model2.weightsPack = model.weightsPack
model2.layersPack = model.layersPack

In [ ]:
str(0.5555)

In [ ]:
pb = ProgressBar(40000, title='Epoch '+str(1), key='outer')
for i in pb:
    pb_inner = ProgressBar(1, title='Inner', key='inner')

In [ ]:
len(X_train)

In [ ]:
from ipy_progressbar import ProgressBar
batch_size = 64
pb = ProgressBar(len(X_train), title='Epoch '+str(i), key='outer')
for i in xrange(10):
    print 'epoch ', i

    for j in xrange(len(X_train)/batch_size):
        train_cost, _ = train_func(X_train[j*batch_size:(j+1)*batch_size], X_train[j*batch_size:(j+1)*batch_size,0], Y_train[j*batch_size:(j+1)*batch_size]) 
        pb_inner = ProgressBar(1, title='Inner', key='inner')

In [ ]:
J_predict = predict_func(X_train[0:1], X_train[0:1, 0])
J_in = NP.concatenate((X_train[0:1, 0:1, :], J_predict[0:1, :-1, :]), axis=1)
v_jacobi = jacobi_func(X_train[0:1], X_train[0:1, 0])

In [ ]:
print J_predict[0, 3]
print Y_train[0, 3]

In [ ]:
print J_predict[0, 3]
print Y_train[0, 3]

In [ ]:
p_jacobi = jacobi_func(X_train[0:1], X_train[0:1, 0])
#print p_jacobi

In [ ]:
NP.shape(v_jacobi)

In [ ]:
NP.sum(abs(v_jacobi[30]), axis=2)

In [ ]:
help(ProgressBar)

In [ ]:
pb = ProgressBar(1000, title='Outer', key='outer')
for i in pb:
    pb_inner = ProgressBar(1, title='Inner', key='inner')
    print 'hhh'